In [1]:
!pip install openai pillow matplotlib

In [5]:
import os
import glob
from openai import OpenAI
import matplotlib.pyplot as plt
from PIL import Image
import base64


# Get API key from environment variable (set in Command Prompt)
api_key = os.getenv("XAI_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please set XAI_API_KEY in your environment variables.")

# Initialize the xAI API client
client = OpenAI(
    api_key=api_key,
    base_url="https://api.x.ai/v1"
)

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return image_file.read()

# Process a few images
root_path = r"D:\LLM\heatmaps"
image_files = glob.glob(os.path.join(root_path, "*_processed.png"))
num_images_to_show = 3

for img_path in image_files[:num_images_to_show]:
    try:
        image_data = encode_image(img_path)
        tumor_type = img_path.split("_")[0].split("\\")[-1]
        
        # Prepare the prompt
        prompt = f"Analyze this MRI heatmap image and provide a detailed caption including the tumor type, location, size, and any irregularities. The image is labeled as a {tumor_type} tumor."
        
        # Make API call
        response = client.chat.completions.create(
            model="grok-vision-beta",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64.b64encode(image_data).decode('utf-8')}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=200
        )

        caption = response.choices[0].message.content
        
        # Display the image and caption
        image = Image.open(img_path).convert("RGB")
        plt.figure(figsize=(8, 6))
        plt.imshow(image)
        plt.title(f"{os.path.basename(img_path)}\nCaption: {caption}")
        plt.axis('off')
        plt.show()
        print(f"{os.path.basename(img_path)}: {caption}")
    except Exception as e:
        print(f"Error processing {os.path.basename(img_path)}: {str(e)}")

Error processing Glioma_enh_1841_processed.png: Error code: 403 - {'code': 'The caller does not have permission to execute the specified operation', 'error': "Your newly created teams doesn't have any credits yet. You can purchase credits on https://console.x.ai/team/bf97a935-b865-4e52-aaf6-91f65c1173b3."}
Error processing Glioma_enh_1842_processed.png: Error code: 403 - {'code': 'The caller does not have permission to execute the specified operation', 'error': "Your newly created teams doesn't have any credits yet. You can purchase credits on https://console.x.ai/team/bf97a935-b865-4e52-aaf6-91f65c1173b3."}
Error processing Glioma_enh_1843_processed.png: Error code: 403 - {'code': 'The caller does not have permission to execute the specified operation', 'error': "Your newly created teams doesn't have any credits yet. You can purchase credits on https://console.x.ai/team/bf97a935-b865-4e52-aaf6-91f65c1173b3."}
